In [2]:
import pandas as pd
import rdkit
from chembl_webresource_client.new_client import new_client

In [67]:
# Создаем API, фильтруя результаты по id таргетной молекулы и типу активности

activity_api= new_client.activity.filter(target_chembl_id='CHEMBL230', type='IC50').only('molecule_chembl_id','canonical_smiles', 'standard_value', 'units', 'standart relation')

molecule_data = list(activity_api)
            
# Создание DataFrame

df = pd.DataFrame(molecule_data)
df.info()

# Проверка SMILES

def smiles_check(smiles_str):
    if rdkit.Chem.MolFromSmiles(smiles_str) is None:
        return False
    else:
        return True

df = df.dropna(subset=['canonical_smiles']) # Удаляем Nan
df = df[df['canonical_smiles'].apply(smiles_check) == True]

# Фильтрация от дубликатов, фильтрация по 'standard_value' и по 'units'

df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')

# Отфильтровываем значения в nM

df1 = df[df['units'] == 'nM']
df1 = df[df['standard_value'] <= 1000]

# Отфильтровываем значения в uM и переводим в nM

df2 = df[df['units'] == 'uM']
df2 = df2[df2['standard_value'] <= 1]
df2['standard_value'] = pd.to_numeric(df2['standard_value'])*1000
df2['units'] = 'nM'

# Отфильтровываем значения в umol/L и переводим в nM

df3 = df[df['units'] == 'umol/L']
df3 = df3[df3['standard_value'] <= 1]
df3['standard_value'] = pd.to_numeric(df3['standard_value'])*1000
df3['units'] = 'nM'

# Отфильтровываем значения в 10'-8M и переводим в nM

df4 = df[df['units'] == "10'-8M"]
df4 = df4[df4['standard_value'] <= 100]
df4['standard_value'] = pd.to_numeric(df3['standard_value'])*10
df4['units'] = 'nM'

# Объединяем и удаляем дубликаты

res = pd.concat([df1, df2, df3, df4], axis = 0)
res = res[(res['standard_value'] != 0) & (res['standard_value'] != '')]
res = pd.DataFrame.drop_duplicates(res, subset='molecule_chembl_id')

# Запись результата в .csv файл

with open('Parsing_res.csv', 'w', newline='', encoding='utf-8'):
    res.to_csv('Parsing_res.csv', columns = ['molecule_chembl_id','canonical_smiles', 'standard_value', 'units'], index = False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7378 entries, 0 to 7377
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   canonical_smiles    7344 non-null   object
 1   molecule_chembl_id  7378 non-null   object
 2   relation            6377 non-null   object
 3   standard_value      6377 non-null   object
 4   units               6396 non-null   object
 5   value               6377 non-null   object
dtypes: object(6)
memory usage: 346.0+ KB
